In [ ]:
import sys
import pandas as pd
from pandas import read_csv
import numpy
from numpy import array
import matplotlib.pyplot as plt
import math
import tensorflow
import keras
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from google.colab import drive
import os

Historia=1
BATCHSIZE=1
EPOCAS=1
LSTM_DIM=1


In [ ]:
print (sys.version_info)
print ("NumPy: ", numpy.__version__)
print ("Pandas: ", pd.__version__)
print ("TensorFlow: ", tensorflow.__version__)
print ("Keras: ", keras.__version__)
print ("SciKitLearn: ", sklearn.__version__)


<h1>Carga del fichero</h1>

In [ ]:

drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/day.csv'
dfBikes=pd.read_csv (path)


dfBikes.rename(columns = {'instant':'Id', 'dteday':'Fecha',
                     "season":'Estacion', 'yr':'Anio', 'mnth':'Mes',
                     'holiday':'Festivo','weekday':'DiaSemana','workingday':'DiaTrabajo','weathersit':'Tiempo','Temperatura':'temp','SensacionTemperatura':'atemp','Humedad':'hum','windspeed':'VelocidadViento','casual':'Casual','registered':'Registrado','cnt':'Cuenta'}, inplace=True)

dfBikes.sort_values(['Id'], inplace=True)
dfBikes.dropna(inplace=True)
dfBikes

## Funciones

In [ ]:
#Convertir los datos en un formato adecuado para entrenar modelos de series temporales, en este caso LSTM
def CreaDatos(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


#funcion para visualizar en un grafico la prediccion y el valor real
#aqui se desnormalizan los datos. Para ello, se despeja la formula de normalizacion
def CalculoRMSE (titulo2):

    testPredict = model.predict(testX, batch_size=BATCHSIZE)*(MaxRealY - MinRealY) + MinRealY
    testReal=testY[:,13]*(MaxRealY - MinRealY) + MinRealY

    testScore = math.sqrt(mean_squared_error(testReal, testPredict.reshape(-1,1)))

    plt.plot (testPredict, label="Predic")
    plt.plot (testReal, label="Real")
    plt.legend ()
    plt.title ("Prediccion de bicicletas alquiladas en el dia.\n"+titulo2+" Historia="+str (Historia)+" Epocas="+str (EPOCAS)+ " LSTM_dim="+str (LSTM_DIM))
    plt.show()
    return testScore








<h1>Procesamiento de los datos</h1>

In [ ]:
#seleccionamos las columnas a utilizar. Todas menos la de la fecha y la del id
df = dfBikes.iloc[:,2:]
print(df)
print(df.shape)

In [ ]:
#buscamos el mayor y menor valor de la clase
MinRealY=df["Cuenta"].min ()
MaxRealY=df["Cuenta"].max ()
print(MinRealY)
print(MaxRealY)

In [ ]:
#escalamos el dataframe
scaler  = MinMaxScaler ()
dataset = scaler.fit_transform(df)
print(dataset)

In [ ]:
#separamos en conjunto de entrenamiento y de test
train_size = int(len(df) * 0.67)
test_size  = len(df) - train_size
train = dataset[0:train_size,:]
test  = dataset[train_size:len(df),:]

<h1>MLP</h1>

In [ ]:
from tensorflow.keras import datasets, layers, models

In [ ]:
#preparamos los datos
trainX=train[:,0:12]
trainY=train[:,13]
testX=test[:,0:12]
testY=test[:,13]


In [ ]:
from contextlib import nullcontext

#dataframe para almacenar las combinaciones de parametros y su RMSE medio obtenido en las 3 iteraciones
dfError = pd.DataFrame(columns=["opt","capas","perdida","batch","epoca", "RMSE"])

#parametros a evaluar y sus valores diferentes
optimizadores=["SGD","adam","RMSprop"]
funciones_perdidas=["mean_squared_error","mean_absolute_error","huber_loss"]
batch_sizes=[16,32,64]
epocas=[6,10,14]
capas=[1,3,5]





#recorremos parametros
for capas_ind in range(len(capas)):
  for perdida_ind in range(len(funciones_perdidas)):
    for batch_ind in range(len(batch_sizes)):
      for epoca_ind in range(len(epocas)):
        for opt_ind in range(len(optimizadores)):

          #imprimir por donde llegamos
          print("\n\n\n")
          print("capas:",str(capas_ind))
          print("perdida:",str(perdida_ind))
          print("batch:",str(batch_ind))
          print("epoca:",str(epoca_ind))
          print("opt:",str(opt_ind))

          #establecemos el optimizador
          optimizer=None
          if(optimizadores[opt_ind]=="SGD"):
            optimizer = keras.optimizers.SGD(learning_rate=1e-3)
            print("SGD")
          if(optimizadores[opt_ind]=="adam"):
            optimizer = keras.optimizers.Adam(learning_rate=1e-3)
            print("Adam")
          if(optimizadores[opt_ind]=="RMSprop"):
            optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
            print("RMSprop")

          #establecemos el modelo
          modelNN_Keras = models.Sequential()
          #aqui input_dim es 12 porque 12 atributos de entrada y salida la dejamos como sigmoid
          #esta capa de entrada se mantiene siempre
          modelNN_Keras.add(layers.Dense(units = 16, activation = 'relu', input_dim = 12))

          #despues añadimos las capas intermedias que correspondan segun el valor del parametro
          for _ in range(capas[capas_ind]):
            modelNN_Keras.add(layers.Dense(units = 6, activation = 'relu'))

          #la capa de salida se mantiene siempre
          modelNN_Keras.add(layers.Dense(units = 1, activation = 'sigmoid'))
          modelNN_Keras.summary()

          #para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteracion
          i=0
          acumulador=0

          #repetimos 3 veces para cada modelo con su combinacion de parametros el proceso de entrenamiento y prediccion
          for i in range(3):
              #ajustamos el optimizador y la funcion de perdida
              modelNN_Keras.compile(optimizer=optimizer, loss=funciones_perdidas[perdida_ind], metrics=['accuracy'])
              #ajustamos las epocas y el batch_size
              history = modelNN_Keras.fit(trainX, trainY, validation_split = 0.1, epochs=epocas[epoca_ind], batch_size=batch_sizes[batch_ind], verbose=0)
              #hacemos prediccion
              y_pred = modelNN_Keras.predict(testX)
              #evaluamos RMSE y lo vamos acumulando
              rmse = math.sqrt(mean_squared_error(testY, y_pred))
              acumulador=acumulador+rmse
              print("ejecuccion: ",str(i))

          #calculamos la media de RMSE en las 3 iteraciones
          media=acumulador/3
          print("media: ",str(media))
          #Lo almacenamos en el dataframe junto a los parametros seleccionados
          dfError.loc[len(dfError)] = [str(optimizadores[opt_ind]),str(capas[capas_ind]),str(funciones_perdidas[perdida_ind]),str(batch_sizes[batch_ind]),str(epocas[epoca_ind]), media]
          print("completado parametros\n\n")




print(dfError)
dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/MLP_resultados.csv", index=False)




<h1>Preparacion de los datos para LSTM</h1>

<h3>Train</h3>

In [ ]:
print(train)
print(train.shape)

In [ ]:
#Convertimos los datos en un formato adecuado para LSTM
trainX, trainY = CreaDatos(train, Historia)

In [ ]:
print(trainX)
print(trainX.shape)

In [ ]:
print(trainY)
print(trainY.shape)

<h3>Test</h3>

In [ ]:
print(test)

In [ ]:
#Convertimos los datos en un formato adecuado para LSTM
testX, testY   = CreaDatos(test, Historia)

In [ ]:
print(testX)
print(testX.shape)

In [ ]:
print(testY)
print (testY.shape)

<h1>LSTM para un problema de regresion</h1>

In [ ]:

#dataframe para almacenar el RMSE con las diferentes combinaciones de parametros
dfError = pd.DataFrame(columns=["func_salida","LSTM_dim","batch","epoca", "RMSE"])

#parametros a evaluar y sus valores diferentes
func_salida=["sigmoid","relu","tanh","linear"]
batch_sizes=[16,32,64]
epocas=[6,10,14]
LSTM_dim=[4,8,12,16]

func_salida_ind=0
batch_ind=0
epocas_ind=0
LSTM_dim_ind=0

#recorremos parametros
for func_salida_ind in range(len(func_salida)):
  for batch_ind in range(len(batch_sizes)):
    for epocas_ind in range(len(epocas)):
      for LSTM_dim_ind in range(len(LSTM_dim)):

        #imprimir por donde llegamos
        print("\n\n\n")
        print("func_salida:",str(func_salida_ind))
        print("LSTM_dim:",str(LSTM_dim_ind))
        print("batch:",str(batch_ind))
        print("epoca:",str(epocas_ind))
        #ajustamos los parametros para cuando vaya a hacer la prediccion en la funcion CalculoRMSE
        BATCHSIZE=batch_sizes[batch_ind]
        EPOCAS=epocas[epocas_ind]
        LSTM_DIM=LSTM_dim[LSTM_dim_ind]

        #creamos el modelo
        model = Sequential()
        #ajustamos parametro lstm_dim
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], input_shape=(trainX.shape[1],trainX.shape[2])))
        #ajustamos parametro funcion de salida
        model.add(Dense(1, activation=func_salida[func_salida_ind]))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.summary()

        #para realizar las 3 iteraciones y acumular el RMSE de cada iteracion
        i=0
        acumulador=0

        #realizamos 3 veces con el modelo el entrenamiento
        for i in range(3):

            #ajustamos el batch_size y el numero de epocas
            model.fit(trainX, trainY[:,13], epochs=epocas[epocas_ind], batch_size=batch_sizes[batch_ind], verbose=0)

            #llamamos a la funcion que obtiene el RMSE y realiza la grafica para comparar los valores reales con los predichos
            testScore=CalculoRMSE ("LSTM-vainilla")
            acumulador=acumulador+testScore
            print("ejecuccion: ",str(i))

        #calculamos la media de RMSE en las 3 iteraciones
        media=acumulador/3
        print("media: ",str(media))
        #Lo almacenamos en el dataframe junto a los parametros seleccionados
        dfError.loc[len(dfError)] = [str(func_salida[func_salida_ind]),str(LSTM_dim[LSTM_dim_ind]),str(batch_sizes[batch_ind]),str(epocas[epocas_ind]), media]
        print("completado parametros\n\n")

#guardamos el dataframe
print(dfError)
dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_1_resultados.csv", index=False)




<h1>LSTM utilizando una ventana temporal</h1>

In [ ]:
#Convertimos los datos en un formato adecuado para LSTM
Historia = 3
trainX, trainY = CreaDatos (train, Historia)
testX, testY   = CreaDatos (test, Historia)

print (trainX.shape, trainY.shape)

In [ ]:

#dataframe para almacenar las combinaciones de parametros y su RMSE medio obtenido en las 3 iteraciones
dfError = pd.DataFrame(columns=["opt","LSTM_dim","perdida","epoca", "RMSE"])

#parametros a evaluar y sus valores diferentes
optimizadores=["SGD","adam","RMSprop"]
funciones_perdidas=["mean_squared_error","mean_absolute_error","huber_loss"]
epocas=[6,10,14]
LSTM_dim=[2,5,10]



LSTM_dim_ind=0
perdida_ind=0
epoca_ind=0
opt_ind=0

#recorremos parametros
for LSTM_dim_ind in range(len(LSTM_dim)):
  for perdida_ind in range(len(funciones_perdidas)):
    for epoca_ind in range(len(epocas)):
        for opt_ind in range(len(optimizadores)):

          #imprimir por donde llegamos
          print("\n\n\n")
          print("LSTM_dim:",str(LSTM_dim_ind))
          print("perdida:",str(perdida_ind))
          print("epoca:",str(epoca_ind))
          print("opt:",str(opt_ind))
          #ajustamos los parametros para cuando vaya a hacer la prediccion en la funcion CalculoRMSE
          BATCHSIZE=1
          EPOCAS=epocas[epoca_ind]
          LSTM_DIM=LSTM_dim[LSTM_dim_ind]

          #establecemos el optimizador
          optimizer=None
          if(optimizadores[opt_ind]=="SGD"):
            optimizer = keras.optimizers.SGD(learning_rate=1e-3)
            print("SGD")
          if(optimizadores[opt_ind]=="adam"):
            optimizer = keras.optimizers.Adam(learning_rate=1e-3)
            print("Adam")
          if(optimizadores[opt_ind]=="RMSprop"):
            optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
            print("RMSprop")

          #creamos el modelo
          model = Sequential()
          #ajustamos parametro LSTM_dim
          model.add(LSTM(LSTM_dim[LSTM_dim_ind], input_shape=(trainX.shape [1], trainX.shape[2])))
          model.add(Dense(1, activation='sigmoid'))
          #ajustamos optimizador y funcion de perdida
          model.compile(loss=funciones_perdidas[perdida_ind], optimizer=optimizer)

          #para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteracion
          i=0
          acumulador=0

          for i in range(3):
              #entrenamos el modelo y ajustamos epocas
              model.fit(trainX, trainY[:,13], epochs=epocas[epoca_ind], batch_size=1, verbose=0)

              #llamamos a la funcion que obtiene el RMSE y realiza la grafica para comparar los valores reales con los predichos
              testScore=CalculoRMSE ("LSTM-vainilla")
              acumulador=acumulador+testScore
              print("ejecuccion: ",str(i))

          #calculamos la media de RMSE en las 3 iteraciones
          media=acumulador/3
          print("media: ",str(media))
          #Lo almacenamos en el dataframe junto a los parametros seleccionados
          dfError.loc[len(dfError)] = [str(optimizadores[opt_ind]),str(LSTM_dim[LSTM_dim_ind]),str(funciones_perdidas[perdida_ind]),str(epocas[epoca_ind]), media]
          print([str(optimizadores[opt_ind]),str(LSTM_dim[LSTM_dim_ind]),str(funciones_perdidas[perdida_ind]),str(epocas[epoca_ind]), media])
          print("completado parametros\n\n")

          #guardamos el dataframe
          print(dfError)
          dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_2_resultados.csv", index=False)




<h1>LSTM apiladas</h1>

In [ ]:
#Convertimos los datos en un formato adecuado para LSTM
historia = 1
trainX, trainY = CreaDatos (train, historia)
testX, testY   = CreaDatos (test, historia)

print (trainX.shape)

In [ ]:
#dataframe para almacenar el RMSE con las diferentes combinaciones de parametros
dfError = pd.DataFrame(columns=["opt","batch","epoca","LSTM_dim", "RMSE"])

#parametros a evaluar y sus valores diferentes
optimizadores=["RMSprop","adam","SGD"]
batch_sizes=[16,32,64]
epocas=[6,10,14]
LSTM_dim=[4,8,12]


func_perdida_ind=0
batch_ind=0
epocas_ind=0
LSTM_dim_ind=0
opt_ind=0

#recorremos parametros

for batch_ind in range(len(batch_sizes)):
  for epocas_ind in range(len(epocas)):
    for LSTM_dim_ind in range(len(LSTM_dim)):
      for opt_ind in range(len(optimizadores)):

        #imprimir por donde llegamos
        print("\n\n\n")
        print("LSTM_dim:",str(LSTM_dim_ind))
        print("batch:",str(batch_ind))
        print("epoca:",str(epocas_ind))
        print("optimizador:",str(opt_ind))
        #ajustamos los parametros para cuando vaya a hacer la prediccion en la funcion CalculoRMSE
        BATCHSIZE=batch_sizes[batch_ind]
        EPOCAS=epocas[epocas_ind]
        LSTM_DIM=LSTM_dim[LSTM_dim_ind]


        #establecemos el optimizador
        optimizer=None
        if(optimizadores[opt_ind]=="SGD"):
          optimizer = keras.optimizers.SGD(learning_rate=1e-3)
          print("SGD")
        if(optimizadores[opt_ind]=="adam"):
          optimizer = keras.optimizers.Adam(learning_rate=1e-3)
          print("Adam")
        if(optimizadores[opt_ind]=="RMSprop"):
          optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
          print("RMSprop")



        #creamos el modelo y entrenamos
        model = Sequential()
        #ajustamos parametro LSTM_Dim
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], input_shape=(trainX.shape[1],trainX.shape[2]), return_sequences=True))
        model.add(LSTM(LSTM_dim[LSTM_dim_ind]))
        model.add(Dense(1, activation='sigmoid'))
        #ajustamos optimizador y funcion de perdida
        model.compile(loss="mean_squared_error", optimizer=optimizer)


        #para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteracion
        i=0
        acumulador=0

        #entrenamos modelo 3 veces
        for i in range(3):
            #ajustamos epocas y batch size
            model.fit(trainX, trainY[:,13], epochs=epocas[epocas_ind], batch_size=batch_sizes[batch_ind], verbose=0)
            #llamamos a la funcion que obtiene el RMSE y realiza la grafica para comparar los valores reales con los predichos
            testScore=CalculoRMSE ("LSTM-dos capas")
            acumulador=acumulador+testScore
            print("ejecuccion: ",str(i))

        #calculamos la media de RMSE en las 3 iteraciones
        media=acumulador/3
        print("media: ",str(media))
        #Lo almacenamos en el dataframe junto a los parametros seleccionados
        dfError.loc[len(dfError)] = [str(optimizadores[opt_ind]),str(batch_sizes[batch_ind]),str(epocas[epocas_ind]),str(LSTM_dim[LSTM_dim_ind]), media]
        print("completado parametros\n\n")

        #guardamos el dataframe
        print(dfError)
        dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_3_resultados.csv", index=False)




<h1>LSTM con memoria entre lote y lote</h1>

In [ ]:
#dataframe para almacenar el RMSE con las diferentes combinaciones de parametros
dfError = pd.DataFrame(columns=["func_act","opt","epoca","LSTM_dim", "RMSE"])

#parametros a evaluar y sus valores diferentes
func_activacion=["sigmoid","relu","tanh","linear"]
optimizadores=["RMSprop","SGD","adam"]
epocas=[6,10,14]
LSTM_dim=[4,8,12]



#recorremos parametros
for func_activacion_ind in range(len(func_activacion)):
  for opt_ind in range(len(optimizadores)):
    for epocas_ind in range(len(epocas)):
      for LSTM_dim_ind in range(len(LSTM_dim)):

        #imprimir por donde llegamos
        print("\n\n\n")
        print("func_acticacion:",str(func_activacion_ind))
        print("LSTM_dim:",str(LSTM_dim_ind))
        print("optimizador:",str(opt_ind))
        print("epoca:",str(epocas_ind))
        #ajustamos los parametros para cuando vaya a hacer la prediccion en la funcion CalculoRMSE
        BATCHSIZE=1
        EPOCAS=epocas[epocas_ind]
        LSTM_DIM=LSTM_dim[LSTM_dim_ind]

        #establecemos el optimizador
        optimizer=None
        if(optimizadores[opt_ind]=="SGD"):
          optimizer = keras.optimizers.SGD(learning_rate=1e-3)
          print("SGD")
        if(optimizadores[opt_ind]=="adam"):
          optimizer = keras.optimizers.Adam(learning_rate=1e-3)
          print("Adam")
        if(optimizadores[opt_ind]=="RMSprop"):
          optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
          print("RMSprop")



        #creamos el modelo y entrenamos
        batch_size_alterado = 1
        model = Sequential()
        #ajustamos parametro LSTM_dim
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], batch_input_shape=(batch_size_alterado, trainX.shape [1], trainX.shape [2]), stateful=True))
        #despues añadimos las capas intermedias que correspondan segun el valor del parametro

        #capa de salida se mantiene
        model.add(Dense(1, activation=func_activacion[func_activacion_ind]))
        #ajusto optimizador
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        #para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteracion
        i=0
        acumulador=0
        k=0

        #repetimos 3 veces el entrenamiento
        for i in range(3):
            #en este modelo, realizamos tantas iteraciones como epocas
            for k in range(epocas[epocas_ind]):
                #ajustamos el parametro de batch
                model.fit(trainX, trainY[:,13], epochs=1, batch_size=1, verbose=0, shuffle=False)

                #llamamos a la funcion que obtiene el RMSE y realiza la grafica para comparar los valores reales con los predichos
                testScore=CalculoRMSE("LSTM - con memoria entre lote y lote")
                acumulador=acumulador+testScore
                print("ejecuccion: ",str(i))

                model.reset_states()


        #calculamos la media de RMSE en las 3 iteraciones
        media=acumulador/3
        print("media: ",str(media))
        #Lo almacenamos en el dataframe junto a los parametros seleccionados
        dfError.loc[len(dfError)] = [str(func_activacion[func_activacion_ind]),str(optimizadores[opt_ind]),str(epocas[epocas_ind]),str(LSTM_dim[LSTM_dim_ind]), media]
        print("completado parametros\n\n")

        #guardamos el dataframe
        print(dfError)
        dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_4_resultados.csv", index=False)




<h1>LSTM's apilados (varias capas) con memoria entre lote y lote</h1>

In [ ]:

#dataframe para almacenar el RMSE con las diferentes combinaciones de parametros
dfError = pd.DataFrame(columns=["func_activacion","opt","epoca","LSTM_dim", "RMSE"])

#parametros a evaluar y sus valores diferentes
func_activacion=["sigmoid","relu","tanh","linear"]
optimizadores=["RMSprop","SGD","adam"]
epocas=[6,10,14]
LSTM_dim=[4,8,12,16]



#recorremos parametros
for func_activacion_ind in range(len(func_activacion)):
  for opt_ind in range(len(optimizadores)):
    for epocas_ind in range(len(epocas)):
      for LSTM_dim_ind in range(len(LSTM_dim)):

        #imprimir por donde llegamos
        print("\n\n\n")
        print("func_activacion:",str(func_activacion_ind))
        print("LSTM_dim:",str(LSTM_dim_ind))
        print("optimizador:",str(opt_ind))
        print("epoca:",str(epocas_ind))
        #ajustamos los parametros para cuando vaya a hacer la prediccion en la funcion CalculoRMSE
        BATCHSIZE=1
        EPOCAS=epocas[epocas_ind]
        LSTM_DIM=LSTM_dim[LSTM_dim_ind]

        #establecemos el optimizador
        optimizer=None
        if(optimizadores[opt_ind]=="SGD"):
          optimizer = keras.optimizers.SGD(learning_rate=1e-3)
          print("SGD")
        if(optimizadores[opt_ind]=="adam"):
          optimizer = keras.optimizers.Adam(learning_rate=1e-3)
          print("Adam")
        if(optimizadores[opt_ind]=="RMSprop"):
          optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
          print("RMSprop")

        #creamos el modelo y entrenamos
        batch_size_alterado = 1
        model = Sequential()
        #ajustamos parametro LSTM_dim
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], batch_input_shape=(batch_size_alterado, trainX.shape[1], trainX.shape[2]), stateful=True, return_sequences=True))
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], batch_input_shape=(batch_size_alterado, trainX.shape[1], trainX.shape[2]), stateful=True))
        #ajustamos parametro func activacion salida
        model.add(Dense(1, activation=func_activacion[func_activacion_ind]))
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        #para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteracion
        i=0
        acumulador=0
        k=0

        #repetimos 3 veces el entreno
        for i in range(3):
            #recorremos bucle tantas epocas tengamos
            for k in range(epocas[epocas_ind]):
                #ajustamos parametro batch_size
                model.fit(trainX, trainY[:,13], epochs=1, batch_size=BATCHSIZE, verbose=0, shuffle=False)
                #llamamos a la funcion que obtiene el RMSE y realiza la grafica para comparar los valores reales con los predichos
                testScore=CalculoRMSE("prueba")
                acumulador=acumulador+testScore
                print("ejecuccion: ",str(i))

                model.reset_states()


        #calculamos la media de RMSE en las 3 iteraciones
        media=acumulador/3
        print("media: ",str(media))
        #Lo almacenamos en el dataframe junto a los parametros seleccionados
        dfError.loc[len(dfError)] = [str(func_activacion[func_activacion_ind]),str(optimizadores[opt_ind]),str(epocas[epocas_ind]),str(LSTM_dim[LSTM_dim_ind]), media]
        print("completado parametros\n\n")

        #guardamos el dataframe
        print(dfError)
        dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/LSTM_5_resultados.csv", index=False)


